In [2]:
!pip install ekphrasis

     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 184kB 7.6MB/s 
     |████████████████████████████████| 71kB 5.2MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-cp37-none-any.whl size=82844 sha256=3d8652a468266ffec134cd23717bed49b41ffc8465fc18b59934fb2791b8770c
  Stored in directory: /root/.cache/pip/wheels/2f/c5/9b/c9b60f535a2cf9fdbc92d84c4801a010c35a9cd348011ed2a1
  Created wheel for ftfy: filename=ftfy-6.0.1-cp37-none-any.whl size=41573 sha256=234185dd193693c5e143e5c3dd5ca249860f040a915808032d837626e436b8b8
  Stored in directory: /root/.cache/pip/wheels/ae/73/c7/9056e14b04919e5c262fe80b54133b1a88d73683d05d7ac65c
Successfully built ekphrasis ftfy


In [3]:
!pip install transformers==4.2.1


     |████████████████████████████████| 1.8MB 6.4MB/s 
     |████████████████████████████████| 870kB 23.5MB/s 
     |████████████████████████████████| 2.9MB 32.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=8ff32af68272e75eef753faa97af8380d47432156e5370ae672a93a0f6ca0d93
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [4]:
pip install tf-models-official


     |████████████████████████████████| 1.1MB 4.9MB/s 
     |████████████████████████████████| 645kB 15.8MB/s 
     |████████████████████████████████| 37.6MB 1.2MB/s 
     |████████████████████████████████| 102kB 9.9MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 174kB 37.2MB/s 
     |████████████████████████████████| 1.2MB 42.4MB/s 
     |████████████████████████████████| 706kB 37.3MB/s 
     |████████████████████████████████| 358kB 45.0MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-cp37-none-any.whl size=22245 sha256=f6f46d13e9377afa8155442cf49f5030a98bfcec7d8cff8726672f7b6f9be64c
  Stored in directory: /root/.cache/pip/wheels/2e/15/f5/aa2a056d223903b52cf4870134e3a01df0c723816835dd08db
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=96c254ae2d0b7b2c3c6c468130428f40a0d17f39fadbfb9ea50ea1a1d5aecf0a
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e62621470

In [5]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
import re
import os
from collections import Counter
from official import nlp
import official.nlp.optimization

import ekphrasis
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

In [6]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.96.133.26:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.96.133.26:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU')]


In [7]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=True,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

Word statistics files not found!
Downloading... 

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [8]:
#Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
%cd '/content/drive/MyDrive/data'
!ls

/content/drive/MyDrive/data
albert-1-b.csv	   albert-2.csv       answer-1-b.csv	    task2.csv
albert-1-b.gsheet  albert-2.gsheet    dev		    test
albert-1-c.csv	   answer-1-a.csv     Dexter_answer.gsheet  train
albert-1-c.gsheet  answer-1-a.gsheet  slang.txt


In [10]:
df_train = pd.read_csv('./train/task_1_b_c.csv', encoding='utf-8')
df_train['humor_rating'] = df_train['humor_rating'].fillna(0)
df_train['humor_controversy'] = df_train['humor_controversy'].fillna(2)
text_train = df_train["text"]
df_train.head(10)

,Unnamed: 0,id,text,is_humor,humor_rating,humor_controversy,offense_rating
0,0,1,TENNESSEE: We're the best state. Nobody even c...,1,2.42,1.0,0.20
1,1,2,A man inserted an advertisement in the classif...,1,2.50,1.0,1.10
2,2,3,How many men does it take to open a can of bee...,1,1.95,0.0,2.40
3,3,4,Told my mom I hit 1200 Twitter followers. She ...,1,2.11,1.0,0.00
4,4,5,Roses are dead. Love is fake. Weddings are bas...,1,2.78,0.0,0.10
5,7,8,ME: I'm such an original. Truly one of a kind....,1,1.79,1.0,0.00
6,11,12,What's the difference between a Mormon man and...,1,2.20,0.0,2.95
7,12,13,Stop calling 9-1-1 because you've run out of t...,1,1.50,1.0,0.00
8,13,14,When you march the streets shouting with peopl...,1,2.16,1.0,0.20
9,17,18,"You can make any sentence creepier by adding ""...",1,1.78,1.0,0.20


In [11]:
df_val = pd.read_csv('./dev/task_1_b_c_dev.csv', encoding='utf-8')
df_val['humor_rating'] = df_val['humor_rating'].fillna(0)
df_val['humor_controversy'] = df_val['humor_controversy'].fillna(2)
text_val = df_val["text"]
df_val.head(10)

,Unnamed: 0,id,text,is_humor,humor_rating,humor_controversy,offense_rating
0,0,8001,What's the difference between a Bernie Sanders...,1,2.45,0.0,1.70
1,1,8002,"Vodka, whisky, tequila. I'm calling the shots.",1,2.00,0.0,0.00
2,2,8003,French people don't masturbate They Jacque off,1,2.95,0.0,1.15
3,3,8004,A lot of Suicide bombers are Muslims - I don't...,1,1.38,0.0,3.75
4,4,8005,What happens when you fingerbang a gypsy on he...,1,2.70,1.0,2.25
5,5,8006,Who's the biggest slut ever? Mrs. Pacman--for ...,1,3.05,1.0,2.10
6,6,8007,Caught a tall black man looking through my win...,1,1.89,0.0,1.80
7,14,8015,Adam came before eve.... but then men normally...,1,3.00,1.0,1.10
8,15,8016,Yeah...sounds good. Let's go into the woods at...,1,2.07,0.0,0.00
9,16,8017,How did the astronaut break up with his girlfr...,1,2.65,0.0,0.05


In [12]:
df_test = pd.read_csv('./test/public_test.csv', encoding='utf-8')
text_test = df_test["text"]
df_test.head(15)

,id,text
0,9001,Finding out your ex got fat is like finding 20...
1,9002,"For Brockmann, stereotypes imperil national se..."
2,9003,A girl runs up to her mother with a pile of cr...
3,9004,gotta wonder if baseball still would've been c...
4,9005,When you're dreading getting in the shower cuz...
5,9006,Parenthood feels like you're on a magical unic...
6,9007,It's Friday night and I'm out of control! Gett...
7,9008,me: I didn't know this was a hibachi restauran...
8,9009,Maybe we need to turn the United States off an...
9,9010,I asked my North Korean friend how it was ther...


In [13]:
print("The length of training data: ",len(text_train))
print("The length of validating data",len(text_val))
print("The length of testing data",len(text_test))

The length of training data:  4932
The length of validating data 632
The length of testing data 1000


In [14]:
def print_text(texts,i,j):
    for u in range(i,j):
        print(texts[u])
        print()

print_text(text_train,0,10)
print("-------------------------------------------------------------------")
print_text(text_val,0,10)

TENNESSEE: We're the best state. Nobody even comes close. *Elevennessee walks into the room* TENNESSEE: Oh shit...

A man inserted an advertisement in the classifieds "Wife Wanted". The next day, he received 1000 of replies, all reading: "You can have mine." Free delivery also available at your door step

How many men does it take to open a can of beer? None. It should be open by the time she brings it to the couch.

Told my mom I hit 1200 Twitter followers. She pointed out how my brother owns a house and I'm wanted by several collection agencies. Oh ma!

Roses are dead. Love is fake. Weddings are basically funerals with cake.

ME: I'm such an original. Truly one of a kind. ALSO ME: [holding a glass of white wine] I love to laugh.

What's the difference between a Mormon man and a Muslim man. A Mormon man gets 72 virgins and THEN kills himself.

Stop calling 9-1-1 because you've run out of toilet paper. Yes. If you're running out of toilet paper don't dial 9-1-1. The number for that is 

In [15]:
# Functions for chat word conversion
f = open("./slang.txt", "r")
chat_words_str = f.read()
chat_words_map_dict = {}
chat_words_list = []

for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("-")[0]
        cw_expanded = line.split("-")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)
print(chat_words_list)
def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

{'ihysmrn  ', 'itzk  ', 'bng  ', '0noe  ', 'h83r  ', 'urg  ', 'ybs  ', 'simcl  ', 'iluvm  ', 'iasb  ', 'ihtp  ', 'y2b  ', 'imhe  ', 'wut  ', 'kool  ', 'ufmf  ', 'mtl  ', 'stfu  ', 'rtfms  ', 'poidnh  ', 'stys  ', 'idgas  ', 'ybbg  ', 'baggkyko  ', 'wthswm  ', 'gttp  ', 'lmbo  ', 'boati  ', 'pleez  ', 'stfd  ', 'yrubm  ', 'oar  ', '8008  ', '2moro  ', 'hcbt1  ', 'ktfo  ', 'wht  ', 'dmu  ', 'tlkn  ', 'urnc  ', 'lez  ', 'ydtm  ', 'stfuysoab  ', 'gky  ', 'gimme  ', 'kthnxbai  ', 'cmbo  ', 'qtpi  ', 'coiwta  ', 'omfg  ', 'trnsltr  ', 'bewbz  ', 'http  ', 'iccl  ', 'yn  ', 'pzled  ', 'luzar  ', '2getha  ', 'plomb  ', 'jka  ', 'chohw  ', 'tff  ', 'idc  ', 'ih  ', 'bfitww  ', 'yhbt  ', 'rofc  ', 'is2g  ', 'dbafwtt  ', 'j4u  ', 'laff  ', 'nade  ', 'tgfitw  ', 'wubmgf  ', 'jtbs  ', 'roxorz  ', 'yfb  ', 'hyk  ', 'omgsh  ', 'ql  ', 'mtc  ', 'clt  ', 'svu  ', 'iois  ', 'dyac  ', 'gjp  ', 'h4xxor  ', 'fyp  ', 'b2b  ', 'yvw  ', 'ihistr  ', 'wbb  ', 'iwsul8r  ', 'joo  ', 'kul  ', 'mins  ', 'lmb  ', 'g

In [16]:
# Train set
text_train = text_train.apply(lambda text: chat_words_conversion(text))
print_text(text_train,0,10)

print("-------------------------------------------------------------------")

# Validation set
text_val = text_val.apply(lambda text: chat_words_conversion(text))
print_text(text_val,0,10)
print("-------------------------------------------------------------------")
# Test set
text_test = text_test.apply(lambda text: chat_words_conversion(text))
# print_text(text_test,0,10)

TENNESSEE: We're the best state. Nobody even comes close. *Elevennessee walks into the room* TENNESSEE: Oh shit...

A man inserted an advertisement in the classifieds "Wife Wanted". The next day, he received 1000 of replies, all reading: "You can have mine." Free delivery also available at your door step

How many men does it take to open a can of beer? None. It should be open by the time she brings it to the couch.

Told my mom I hit 1200 Twitter followers. She pointed out how my brother owns a house and I'm wanted by several collection agencies. Oh ma!

Roses are dead. Love is fake. Weddings are basically funerals with cake.

ME: I'm such an original. Truly one of a kind. ALSO ME: [holding a glass of white wine] I love to laugh.

What's the difference between a Mormon man and a Muslim man. A Mormon man gets 72 virgins and THEN kills himself.

Stop calling 9-1-1 because you've run out of toilet paper. Yes. If you're running out of toilet paper don't dial 9-1-1. The number for that is 

In [17]:
def ekphrasis_pipe(sentence):
    cleaned_sentence = " ".join(text_processor.pre_process_doc(sentence))
    return cleaned_sentence
# Training set
text_train = text_train.apply(lambda text: ekphrasis_pipe(text))
print("Training set completed.......")
#Validation set
text_val = text_val.apply(lambda text: ekphrasis_pipe(text))
print("Validation set completed.......")
#Test set
text_test = text_test.apply(lambda text: ekphrasis_pipe(text))
print("Test set completed.......")

Training set completed.......
Validation set completed.......
Test set completed.......


In [20]:
# Finding length of longest array
maxLen = len(max(text_test,key = lambda text: len(text.split(" "))).split(" "))
print(maxLen)

74


In [21]:
u = lambda text: len(text.split(" "))
sentence_lengths = []
for x in text_test:
    sentence_lengths.append(u(x))
print(sorted(sentence_lengths)[-200:])
print(len(sentence_lengths))

[38, 38, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 44, 44, 44, 44, 44, 44, 44, 44, 44, 45, 45, 45, 45, 46, 46, 46, 46, 46, 46, 46, 47, 47, 47, 47, 47, 47, 47, 47, 48, 48, 48, 48, 48, 48, 49, 49, 49, 49, 49, 50, 50, 50, 50, 50, 50, 51, 51, 51, 51, 52, 52, 52, 52, 52, 52, 52, 52, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 54, 54, 54, 54, 54, 54, 55, 55, 55, 55, 55, 56, 56, 56, 56, 56, 57, 57, 57, 57, 57, 57, 58, 58, 58, 58, 59, 59, 59, 59, 59, 59, 60, 60, 60, 60, 61, 61, 62, 62, 62, 63, 63, 63, 63, 64, 64, 64, 64, 64, 64, 65, 65, 65, 65, 66, 66, 66, 67, 67, 68, 69, 69, 73, 73, 74]
1000


In [22]:
is_humor = df_train["is_humor"]
humor_rating = df_train["humor_rating"]
humor_controversy = df_train["humor_controversy"].astype(int)
offense_rating = df_train["offense_rating"]
print(Counter(is_humor))
print(Counter(humor_controversy))

Counter({1: 4932})
Counter({0: 2467, 1: 2465})


In [23]:
is_humor_val = df_val["is_humor"]
humor_rating_val = df_val["humor_rating"]
humor_controversy_val = df_val["humor_controversy"].astype(int)
offense_rating_val = df_val["offense_rating"]
print(Counter(is_humor_val))
print(Counter(humor_controversy_val))

Counter({1: 632})
Counter({0: 324, 1: 308})


In [24]:
from transformers import RobertaTokenizerFast, TFRobertaModel, TFBertModel, BertTokenizerFast, ElectraTokenizerFast, TFElectraModel, AlbertTokenizerFast, TFAlbertModel, XLNetTokenizerFast, TFXLNetModel, MPNetTokenizerFast, TFMPNetModel
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
# Use tokenizer as required
tokenizer = AlbertTokenizerFast.from_pretrained('albert-large-v2')
text_train = list(text_train)
text_val = list(text_val)
text_test = list(text_test)
train_encodings = tokenizer(text_train, max_length=150, truncation=True, padding="max_length", return_tensors='tf')
val_encodings = tokenizer(text_val, max_length=150, truncation=True, padding="max_length", return_tensors='tf')
test_encodings = tokenizer(text_test, max_length=150, truncation=True, padding="max_length", return_tensors='tf')
print(np.shape(train_encodings["input_ids"]))
print(np.shape(val_encodings["input_ids"]))
print(np.shape(test_encodings["input_ids"]))
print("***************************************************************************")
print(val_encodings["input_ids"][0])


(4932, 150)
(632, 150)
(1000, 150)
***************************************************************************
tf.Tensor(
[    2    98    13    22    13    18    14  2841   128    21 17724 11574
 10067    17    21  4211 27600    13    60    21  4211 27600  1121  3049
    20    14 16460    13     9     3     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
 

In [25]:
def hahackathon_task_1(input_shape):
    # Import model as required
    model = TFAlbertModel.from_pretrained('albert-large-v2')
    layer = model.layers[0]
    #Model
    inputs = keras.Input(shape=input_shape, dtype='int32')
    input_masks = keras.Input(shape=input_shape, dtype='int32')

    outputs = layer([inputs, input_masks])
    output = outputs[0]
    # pooled_output = outputs[1]
    pooled_output = output[:, 0, :] # Use for bert, roberta, albert, mpnet, electra
    # pooled_output = output[:, -1] #Use for XLNet

    # Humour regression
    humor_reg = layers.Dropout(0.3)(pooled_output)
    humor_reg = layers.Dense(1)(humor_reg)

    model = keras.Model(inputs=[inputs,input_masks], outputs=humor_reg, name='task_1_b')
    
    return model

In [26]:
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    model = hahackathon_task_1((150,))
    optimizer = keras.optimizers.Adam(learning_rate=3e-5)
    loss_fun = [
          tf.keras.losses.LogCosh()
    ]
    metric = [
        tf.keras.metrics.RootMeanSquaredError()
    ]
    model.compile(optimizer=optimizer, loss=loss_fun, metrics=metric)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Some layers from the model checkpoint at albert-large-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFAlbertModel were initialized from the model checkpoint at albert-large-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


In [27]:
model.summary()

Model: "task_1_b"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 150)]        0                                            
__________________________________________________________________________________________________
albert (TFAlbertMainLayer)      TFBaseModelOutputWit 17683968    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 1024)         0           albert[0][0]              

In [28]:
checkpoint = ModelCheckpoint(filepath='/content/task-1-b-albert.{epoch:03d}.h5',
                                 verbose = 0,
                                 save_weights_only=True,
                                 epoch=4)

In [29]:
# Albert base pooled 
history_task_1_b = model.fit(
    x = [train_encodings["input_ids"], train_encodings["attention_mask"]],
    y = humor_rating,
    validation_data = ([val_encodings["input_ids"],val_encodings["attention_mask"]], humor_rating_val),
    callbacks = [checkpoint],
    batch_size=16,
    shuffle=True,
    epochs=4)

Epoch 1/4


309/309 [==============================] - 112s 136ms/step - loss: 0.3328 - root_mean_squared_error: 0.9434 - val_loss: 0.1333 - val_root_mean_squared_error: 0.5486
Epoch 2/4
309/309 [==============================] - 28s 90ms/step - loss: 0.1596 - root_mean_squared_error: 0.6058 - val_loss: 0.1259 - val_root_mean_squared_error: 0.5319
Epoch 3/4
309/309 [==============================] - 28s 90ms/step - loss: 0.1395 - root_mean_squared_error: 0.5628 - val_loss: 0.1223 - val_root_mean_squared_error: 0.5235
Epoch 4/4
309/309 [==============================] - 28s 91ms/step - loss: 0.1279 - root_mean_squared_error: 0.5386 - val_loss: 0.1209 - val_root_mean_squared_error: 0.5213


In [31]:
from sklearn.metrics import mean_squared_error
from math import sqrt
import time
start_time = time.time()
val_answer = model.predict([val_encodings["input_ids"],val_encodings["attention_mask"]])
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.2582614421844482 seconds ---


In [ ]:

print("mean_squared_error: ")
sqrt(mean_squared_error(humor_rating_val, val_answer))


In [ ]:
import time
start_time = time.time()
test_answer = model.predict([test_encodings["input_ids"],test_encodings["attention_mask"]])

In [ ]:
test_1_b = []
for i in range(0,len(test_answer)):
  test_1_b.append(test_answer[i][0])
test_id = df_test["id"]

test_dict = {
    "id" : test_id,
    "humor_rating" : test_1_b
}
df_test = pd.DataFrame(test_dict)
df_test.head()

In [ ]:
df_test.to_csv('answer-1-b.csv', index=False)